In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import mplhep as hep
from matplotlib import colors

from boostedhh import utils, hh_vars

In [ ]:
# automatically reloads imported files on edits
%load_ext autoreload
%autoreload 2

In [ ]:
MAIN_DIR = Path("../../../")

year = "2023"

exclude = {"highptMu": False}

plot_dir = MAIN_DIR / f"plots/TriggerStudy/25Apr7/{year}"
plot_dir.mkdir(parents=True, exist_ok=True)

samples_dir = {
    "2022": "/ceph/cms/store/user/rkansal/bbtautau/skimmer/24Nov21ParTMass_v12_private_signal",
    "2022EE": "/ceph/cms/store/user/rkansal/bbtautau/skimmer/25Jan22AddYears_v12_private_signal",
    "2023": "/ceph/cms/store/user/lumori/bbtautau/skimmer/25Mar7Signal_v12_private_signal",
    "2023BPix": "/ceph/cms/store/user/lumori/bbtautau/skimmer/25Mar7Signal_v12_private_signal",
}[year]

sig = {
    "2022EE": "GluGlutoHHto2B2Tau_kl-1p00_kt-1p00_c2-0p00_TuneCP5_13p6TeV_powheg-pythia8",
    "2022": "GluGlutoHHto2B2Tau_kl-1p00_kt-1p00_c2-0p00_LHEweights_TuneCP5_13p6TeV_powheg-pythia8",
    "2023": "GluGlutoHHto2B2Tau_kl-1p00_kt-1p00_c2-0p00_TuneCP5_13p6TeV_powheg-pythia8",
    "2023BPix": "GluGlutoHHto2B2Tau_kl-1p00_kt-1p00_c2-0p00_TuneCP5_13p6TeV_powheg-pythia8",
}[year]

In [ ]:
df = pd.read_parquet(f"{samples_dir}/{year}/{sig}/parquet/")

In [ ]:
df4b = pd.read_parquet(f"{samples_dir}/{year}/{sig}/parquet")

In [ ]:
cf = utils.get_cutflow(
    f"{samples_dir}/{year}/{sig}/pickles",
    year,
    sig,
)

In [ ]:
cf4b = utils.get_cutflow(
    f"{samples_dir}/{year}/{sig}/pickles/",
    year,
    sig,
)

In [ ]:
# list(df.columns)

In [ ]:
cf["ak8_pt"] / cf["all"]

In [ ]:
cf4b["ak8_pt"] / cf4b["all"]

In [ ]:
np.mean((df["ak8FatJetPt"][1] > 250))

In [ ]:
np.mean((df4b["ak8FatJetPt"][1] > 230))

In [ ]:
np.sum((df["ak8FatJetPt"][1] > 250) * df["GenTauhh"][0]) / np.sum(df["GenTauhh"][0])

In [ ]:
np.sum((df["ak8FatJetPt"][1] > 230) * (df["ak8FatJetPt"][0] > 250)) / np.sum(
    (df["ak8FatJetPt"][0] > 250)
)

In [ ]:
for cat in ["GenTauhh", "GenTauhmu", "GenTauhe"]:
    print(
        np.sum((df["ak8FatJetPt"][0] > 250) * df[cat][0])
        / np.sum(df[cat][0])
        * (cf["ak8_pt"] / cf["all"])
    )

In [ ]:
for cat in ["GenTauhh", "GenTauhmu", "GenTauhe"]:
    print(
        np.sum((df["ak8FatJetPt"][1] > 200) * (df["ak8FatJetPt"][0] > 250) * df[cat][0])
        / np.sum((df[cat][0]) * (df["ak8FatJetPt"][0] > 250))
    )

In [ ]:
bbbb_triggered = (
    df4b["HLT_AK8PFJet250_SoftDropMass40_PFAK8ParticleNetBB0p35"][0]
    | df4b["HLT_AK8PFJet425_SoftDropMass40"][0]
)
higgs = utils.make_vector(df4b, name="GenHiggs")
mhh = (higgs[:, 0] + higgs[:, 1]).mass
weights = df4b["weight"][0]

twoboosted = df4b["ak8FatJetPt"][1] > 300

print(np.mean(bbbb_triggered))
print(np.sum(bbbb_triggered * twoboosted) / np.sum((twoboosted)))

In [ ]:
xsecbr = 34.13 * 0.07 * 0.88
p250eff = 0.086 * (((0.3 * 0.42) + 2 * (0.23 * 0.23)) / 0.88)
trigeff = 0.94
print(p250eff)
print(xsecbr * p250eff * trigeff)

In [ ]:
p230eff = 0.086 * (((0.38 * 0.42) + 2 * (0.23 * 0.3)) / 0.88)
trigeff = 0.92
print(p230eff)
print(xsecbr * p230eff * trigeff)

In [ ]:
p200eff = 0.086 * (((0.38 * 0.516) + 2 * (0.23 * 0.41)) / 0.88)
trigeff = 0.89
print(p200eff)
print(xsecbr * p200eff * trigeff)

In [ ]:
weights = df["weight"][0]
higgs = utils.make_vector(df, name="GenHiggs")
mhh = (higgs[:, 0] + higgs[:, 1]).mass
hbbpt = higgs[df["GenHiggsChildren"] == 5].pt
httpt = higgs[df["GenHiggsChildren"] == 15].pt

In [ ]:
# bb_mask = df["GenHiggsDecayMode"] == 1

## Triggers

In [ ]:
# Tau triggers Run 3 https://twiki.cern.ch/twiki/bin/view/CMSPublic/Run3TauHLT

# 2022 and 2022EE


def triggers_2022():

    t22 = {}
    t22["pnetbb"] = df["HLT_AK8PFJet250_SoftDropMass40_PFAK8ParticleNetBB0p35"][0]
    t22["pnettt"] = df["HLT_AK8PFJet230_SoftDropMass40_PFAK8ParticleNetTauTau0p30"][0]
    t22["pnet"] = t22["pnetbb"] | t22["pnettt"]

    t22["singletau"] = df["HLT_LooseDeepTauPFTauHPS180_L2NN_eta2p1"][0]

    t22["pfjet"] = df["HLT_AK8PFJet425_SoftDropMass40"][0]

    t22["quadjet"] = (
        df["HLT_QuadPFJet70_50_40_35_PFBTagParticleNet_2BTagSum0p65"][0]
        | df["HLT_QuadPFJet103_88_75_15_PFBTagDeepJet_1p3_VBF2"][0]
        | df["HLT_QuadPFJet103_88_75_15_DoublePFBTagDeepJet_1p3_7p7_VBF1"][0]
    )

    t22["ditau"] = (
        df["HLT_DoubleMediumDeepTauPFTauHPS35_L2NN_eta2p1"][0]
        | df["HLT_DoubleMediumDeepTauPFTauHPS30_L2NN_eta2p1_PFJet60"][0]
        | df["HLT_DoubleMediumDeepTauPFTauHPS30_L2NN_eta2p1_PFJet75"][0]
    )

    t22["singlemuon"] = (
        df["HLT_IsoMu24"][0] if exclude["highptMu"] else df["HLT_IsoMu24"][0] | df["HLT_Mu50"][0]
    )

    t22["mutau"] = (
        df["HLT_IsoMu20_eta2p1_LooseDeepTauPFTauHPS27_eta2p1_CrossL1"][0]
        | df["HLT_IsoMu24_eta2p1_LooseDeepTauPFTauHPS180_eta2p1"][0]
        | df["HLT_IsoMu24_eta2p1_LooseDeepTauPFTauHPS30_eta2p1_CrossL1"][0]
        | df["HLT_IsoMu24_eta2p1_MediumDeepTauPFTauHPS35_L2NN_eta2p1_CrossL1"][0]
        | df["HLT_IsoMu24_eta2p1_MediumDeepTauPFTauHPS30_L2NN_eta2p1_CrossL1"][0]
        | df["HLT_IsoMu24_eta2p1_MediumDeepTauPFTauHPS45_L2NN_eta2p1_CrossL1"][0]
        | df["HLT_IsoMu20_eta2p1_TightChargedIsoPFTauHPS27_eta2p1_CrossL1"][0]
        | df["HLT_IsoMu20_eta2p1_TightChargedIsoPFTauHPS27_eta2p1_TightID_CrossL1"][0]
    )

    t22["egamma"] = (
        df["HLT_Ele30_WPTight_Gsf"][0]
        | df["HLT_Ele115_CaloIdVT_GsfTrkIdT"][0]
        | df["HLT_Ele50_CaloIdVT_GsfTrkIdT_PFJet165"][0]
        | df["HLT_Photon200"][0]
    )

    t22["etau"] = (
        df["HLT_Ele24_eta2p1_WPTight_Gsf_LooseDeepTauPFTauHPS30_eta2p1_CrossL1"][0]
        | df["HLT_Ele24_eta2p1_WPTight_Gsf_TightChargedIsoPFTauHPS30_eta2p1_CrossL1"][0]
    )

    return t22


def triggers_2023():

    t23 = {}

    t23["pnetbb"] = df["HLT_AK8PFJet230_SoftDropMass40_PNetBB0p06"][0]
    t23["pnettt"] = df["HLT_AK8PFJet230_SoftDropMass40_PNetTauTau0p03"][0]
    t23["pnet"] = t23["pnetbb"] | t23["pnettt"]

    t23["pfjet_legacy"] = df["HLT_AK8PFJet425_SoftDropMass40"][0]
    t23["pfjet_lowmass"] = df["HLT_AK8PFJet420_MassSD30"][0]
    t23["pfjet"] = t23["pfjet_legacy"] | t23["pfjet_lowmass"]

    t23["pnetbb_lowpt"] = df[
        "HLT_AK8PFJet220_SoftDropMass40_PNetBB0p06_DoubleAK4PFJet60_30_PNet2BTagMean0p50"
    ][0]

    t23["pk_quadjet_loose"] = df["HLT_PFHT280_QuadPFJet30_PNet2BTagMean0p55"][0]
    t23["pk_quadjet"] = df["HLT_PFHT340_QuadPFJet70_50_40_40_PNet2BTagMean0p70"][0]

    t23["quadjet"] = (
        df["HLT_QuadPFJet103_88_75_15_PFBTagDeepJet_1p3_VBF2"][0]
        | df["HLT_QuadPFJet103_88_75_15_DoublePFBTagDeepJet_1p3_7p7_VBF1"][0]
    )

    t23["singletau"] = df["HLT_LooseDeepTauPFTauHPS180_L2NN_eta2p1"][0]
    t23["singlemuon"] = df["HLT_IsoMu24"][0] | df["HLT_Mu50"][0]

    t23["ditau"] = (
        df["HLT_DoubleMediumDeepTauPFTauHPS35_L2NN_eta2p1"][0]
        | df["HLT_DoubleMediumDeepTauPFTauHPS30_L2NN_eta2p1_PFJet60"][0]
        | df["HLT_DoubleMediumDeepTauPFTauHPS30_L2NN_eta2p1_PFJet75"][0]
    )

    t23["mutau"] = (
        df["HLT_IsoMu20_eta2p1_LooseDeepTauPFTauHPS27_eta2p1_CrossL1"][0]
        | df["HLT_IsoMu24_eta2p1_LooseDeepTauPFTauHPS180_eta2p1"][0]
        | df["HLT_IsoMu24_eta2p1_LooseDeepTauPFTauHPS30_eta2p1_CrossL1"][0]
        | df["HLT_IsoMu24_eta2p1_MediumDeepTauPFTauHPS35_L2NN_eta2p1_CrossL1"][0]
        | df["HLT_IsoMu24_eta2p1_MediumDeepTauPFTauHPS30_L2NN_eta2p1_CrossL1"][0]
        | df["HLT_IsoMu24_eta2p1_MediumDeepTauPFTauHPS45_L2NN_eta2p1_CrossL1"][0]
    )

    t23["egamma"] = (
        df["HLT_Ele30_WPTight_Gsf"][0]
        | df["HLT_Ele115_CaloIdVT_GsfTrkIdT"][0]
        | df["HLT_Ele50_CaloIdVT_GsfTrkIdT_PFJet165"][0]
        | df["HLT_Photon200"][0]
    )

    t23["etau"] = df["HLT_Ele24_eta2p1_WPTight_Gsf_LooseDeepTauPFTauHPS30_eta2p1_CrossL1"][0]

    return t23

In [ ]:
def get_triggers_dict(trs):
    triggers_dict = {
        "tauhh": {
            "mask": df["GenTauhh"][0],
            "label": r"$\tau_h\tau_h$",
            "triggers": {
                "All": trs["pnet"]
                | trs["pfjet"]
                | trs["quadjet"]
                | trs["singletau"]
                | trs["ditau"],
                "PNetBB": trs["pnetbb"],
                "PNetTauTau": trs["pnettt"],
                "PNet": trs["pnet"],
                "PFJet": trs["pfjet"],
                "PNet | SingleTau| Di-tau": trs["pnet"] | trs["singletau"] | trs["ditau"],
                "PNet | PFJet | Quad-jet": trs["pnet"] | trs["pfjet"] | trs["quadjet"],
                "PNet | PFJet | Quad-jet | SingleTau": trs["pnet"]
                | trs["pfjet"]
                | trs["quadjet"]
                | trs["singletau"],
                "PNet | PFJet | Quad-jet | Di-tau": trs["pnet"]
                | trs["pfjet"]
                | trs["quadjet"]
                | trs["ditau"],
                "PNet | PFJet | SingleTau| Di-tau": trs["pnet"]
                | trs["pfjet"]
                | trs["singletau"]
                | trs["ditau"],
            },
        },
        "tauhh_minus": {
            "mask": df["GenTauhh"][0],
            "label": r"$\tau_h\tau_h$",
            "triggers": {
                "All": trs["pnet"]
                | trs["pfjet"]
                | trs["quadjet"]
                | trs["singletau"]
                | trs["ditau"],
                "-PNetBB": trs["pnettt"]
                | trs["pfjet"]
                | trs["quadjet"]
                | trs["singletau"]
                | trs["ditau"],
                "-PNetTauTau": trs["pnetbb"]
                | trs["pfjet"]
                | trs["quadjet"]
                | trs["singletau"]
                | trs["ditau"],
                "-PFJet": trs["pnet"] | trs["quadjet"] | trs["singletau"] | trs["ditau"],
                "-Quad-jet": trs["pnet"] | trs["pfjet"] | trs["singletau"] | trs["ditau"],
                "-SingleTau": trs["pnet"] | trs["pfjet"] | trs["quadjet"] | trs["ditau"],
                "-Di-tau": trs["pnet"] | trs["pfjet"] | trs["quadjet"] | trs["singletau"],
            },
        },
        "tauhmu": {
            "mask": df["GenTauhmu"][0],
            "label": r"$\tau_h\mu$",
            "triggers": {
                "All": trs["pnet"]
                | trs["singlemuon"]
                | trs["mutau"]
                | trs["singletau"]
                | trs["ditau"]
                | trs["pfjet"],
                "PNetBB": trs["pnetbb"],
                "PNetTauTau": trs["pnettt"],
                "PNetBB | TauTau": trs["pnet"],
                "Muon": trs["singlemuon"],
                "Mu-tau": trs["mutau"],
                "SingleTau": trs["singletau"],
                "Di-tau": trs["ditau"],
                "PFJet": trs["pfjet"],
            },
        },
        "tauhmu_minus": {
            "mask": df["GenTauhmu"][0],
            "label": r"$\tau_h\mu$",
            "triggers": {
                "All": trs["pnet"]
                | trs["singlemuon"]
                | trs["mutau"]
                | trs["singletau"]
                | trs["ditau"]
                | trs["pfjet"],
                "-PNetBB": trs["pnettt"]
                | trs["singlemuon"]
                | trs["mutau"]
                | trs["singletau"]
                | trs["ditau"]
                | trs["pfjet"],
                "-PNetTauTau": trs["pnetbb"]
                | trs["singlemuon"]
                | trs["mutau"]
                | trs["singletau"]
                | trs["ditau"]
                | trs["pfjet"],
                "-Muon": trs["pnet"]
                | trs["mutau"]
                | trs["singletau"]
                | trs["ditau"]
                | trs["pfjet"],
                "-Mu-tau": trs["pnet"]
                | trs["singlemuon"]
                | trs["singletau"]
                | trs["ditau"]
                | trs["pfjet"],
                "-SingleTau": trs["pnet"]
                | trs["singlemuon"]
                | trs["mutau"]
                | trs["ditau"]
                | trs["pfjet"],
                "-Di-tau": trs["pnet"]
                | trs["singlemuon"]
                | trs["mutau"]
                | trs["singletau"]
                | trs["pfjet"],
                "-PFJet": trs["pnet"]
                | trs["singlemuon"]
                | trs["mutau"]
                | trs["singletau"]
                | trs["ditau"],
            },
        },
        "tauhe": {
            "mask": df["GenTauhe"][0],
            "label": r"$\tau_he$",
            "triggers": {
                "All": trs["pnet"]
                | trs["egamma"]
                | trs["etau"]
                | trs["singletau"]
                | trs["ditau"]
                | trs["pfjet"],
                "PNetBB": trs["pnetbb"],
                "PNetTauTau": trs["pnettt"],
                "PNetBB | TauTau": trs["pnet"],
                "EGamma": trs["egamma"],
                "e-tau": trs["etau"],
                "SingleTau": trs["singletau"],
                "Di-tau": trs["ditau"],
            },
        },
        "tauhe_minus": {
            "mask": df["GenTauhe"][0],
            "label": r"$\tau_he$",
            "triggers": {
                "All": trs["pnet"]
                | trs["egamma"]
                | trs["etau"]
                | trs["singletau"]
                | trs["ditau"]
                | trs["pfjet"],
                "-PNetBB": trs["pnettt"]
                | trs["egamma"]
                | trs["etau"]
                | trs["singletau"]
                | trs["ditau"]
                | trs["pfjet"],
                "-PNetTauTau": trs["pnetbb"]
                | trs["egamma"]
                | trs["etau"]
                | trs["singletau"]
                | trs["ditau"]
                | trs["pfjet"],
                "-EGamma": trs["pnet"]
                | trs["etau"]
                | trs["singletau"]
                | trs["ditau"]
                | trs["pfjet"],
                "-e-tau": trs["pnet"]
                | trs["egamma"]
                | trs["singletau"]
                | trs["ditau"]
                | trs["pfjet"],
                "-SingleTau": trs["pnet"]
                | trs["egamma"]
                | trs["etau"]
                | trs["ditau"]
                | trs["pfjet"],
                "-Di-tau": trs["pnet"]
                | trs["egamma"]
                | trs["etau"]
                | trs["singletau"]
                | trs["pfjet"],
                "-PFJet": trs["pnet"]
                | trs["egamma"]
                | trs["etau"]
                | trs["singletau"]
                | trs["ditau"],
            },
        },
    }
    if year in ["2023", "2023BPix"]:
        triggers_dict["tauhh"]["triggers"].update(
            {
                "All": trs["pnet"]
                | trs["pnetbb_lowpt"]
                | trs["pfjet"]
                | trs["quadjet"]
                | trs["singletau"]
                | trs["ditau"]
                | trs["pk_quadjet_loose"],
                "PNet | Parking ": trs["pnet"] | trs["pk_quadjet_loose"],
                "Parking Quad-jet": trs["pk_quadjet_loose"],
                "PNetBB (low pt)": trs["pnetbb_lowpt"],
            }
        )

        for key in triggers_dict["tauhh_minus"]["triggers"].keys():
            triggers_dict["tauhh_minus"]["triggers"][key] = (
                triggers_dict["tauhh_minus"]["triggers"][key] | trs["pk_quadjet_loose"]
            )

        triggers_dict["tauhh_minus"]["triggers"].update(
            {
                "-Parking Quad-jet": trs["pnet"]
                | trs["pfjet"]
                | trs["quadjet"]
                | trs["singletau"]
                | trs["ditau"],
            }
        )
        triggers_dict["tauhmu"]["triggers"].update(
            {
                "PNetBB (low pt)": trs["pnetbb_lowpt"],
            }
        )
        triggers_dict["tauhh_minus"]["triggers"].update(
            {
                "PNetBB (low pt)": trs["pnetbb_lowpt"],
            }
        )
    return triggers_dict

In [ ]:
triggers = triggers_2022() if year in ["2022", "2022EE"] else triggers_2023()
triggers_dict = get_triggers_dict(triggers)

In [ ]:
plt.rcParams.update({"font.size": 14})

plot_vars = [
    (mhh, "mhh", r"$m_{HH}$ [GeV]", np.linspace(250, 1500, 30)),
    (hbbpt, "hbbpt", r"Hbb $p_{T}$ [GeV]", np.linspace(230, 500, 20)),
    (httpt, "httpt", r"H$\tau\tau$ $p_{T}$ [GeV]", np.linspace(230, 500, 20)),
]

for cat, vals in triggers_dict.items():
    for kinvar, kinname, kinlabel, bins in plot_vars:
        (mask, label, triggers) = vals.values()

        fig, (ax, rax) = plt.subplots(
            2, 1, figsize=(9, 11), gridspec_kw=dict(height_ratios=[4, 1], hspace=0.07), sharex=True
        )

        hists = {"Preselection": np.histogram(kinvar[mask], bins=bins, weights=weights[mask])}
        ratios = {}

        hep.histplot(
            hists["Preselection"],
            yerr=False,
            label="Preselection",
            ax=ax,
        )

        colours = plt.cm.tab20.colors[1:]

        for key, c in zip(triggers.keys(), colours):
            # print(key)
            hists[key] = np.histogram(
                kinvar[mask & triggers[key]], bins=bins, weights=weights[mask & triggers[key]]
            )
            ratios[key] = hists[key][0] / hists["Preselection"][0]

            hep.histplot(
                hists[key],
                yerr=False,
                label=key,
                ax=ax,
                color=c,
            )

            hep.histplot(
                (ratios[key], bins),
                yerr=False,
                label=key,
                ax=rax,
                histtype="errorbar",
                color=c,
                # markersize=20,
                linestyle="--",
                # markeredgewidth=3,
                # fillstyle='none',
            )

        ax.set_ylabel("Events [A.U.]")
        ax.legend()
        ax.set_title(label)
        ax.set_xlim(bins[0], bins[-1])
        ax.set_ylim(0)

        # rax.legend()
        rax.grid(axis="y")
        rax.set_xlabel(kinlabel)
        rax.set_ylabel("Triggered / Preselection")

        ylims = [0.5, 1] if (cat.endswith("minus") and kinname != "mhh") else [0, 1]
        rax.set_ylim(ylims)

        hep.cms.label(ax=ax, data=False, year=year, com="13.6")

        plt.savefig(plot_dir / f"{kinname}_{cat}.pdf", bbox_inches="tight")
        plt.savefig(plot_dir / f"{kinname}_{cat}.png", bbox_inches="tight")
        plt.show()

In [ ]:
from boostedhh.utils import PAD_VAL

tvars = {}
qcdouts = ["QCD0HF", "QCD1HF", "QCD2HF"]  # HF = heavy flavor = {c,b}
topouts = ["TopW", "TopbW"]  # "TopbWev", "TopbWmv", "TopbWtauhv", "TopbWq", "TopbWqq"]
tvars["PQCD"] = sum([df[f"ak8FatJetParT{k}"] for k in qcdouts]).to_numpy()
tvars["PTop"] = sum([df[f"ak8FatJetParT{k}"] for k in topouts]).to_numpy()
tvars["XbbvsQCD"] = np.nan_to_num(
    df[f"ak8FatJetParTXbb"] / (df[f"ak8FatJetParTXbb"] + tvars["PQCD"]),
    nan=PAD_VAL,
)
tvars["XbbvsQCDTop"] = np.nan_to_num(
    df["ak8FatJetParTXbb"] / (df["ak8FatJetParTXbb"] + tvars["PQCD"] + tvars["PTop"]),
    nan=PAD_VAL,
)

In [ ]:
cats = [cat for cat in triggers_dict.keys() if not cat.endswith("minus")]
boostedsels = {
    "1 boosted jet (> 250)": df["ak8FatJetPt"][0] > 250,
    "2 boosted jets (> 250)": (df["ak8FatJetPt"][0] > 250) & (df["ak8FatJetPt"][1] > 250),
    "2 boosted jets (>250, >230)": (df["ak8FatJetPt"][0] > 250) & (df["ak8FatJetPt"][1] > 230),
    "2 boosted jets (>250, >200)": (df["ak8FatJetPt"][0] > 250) & (df["ak8FatJetPt"][1] > 200),
    "2 boosted jets (>250, >200), XbbvsQCD > 0.95": (df["ak8FatJetPt"][0] > 250)
    & (df["ak8FatJetPt"][1] > 200)
    & (tvars["XbbvsQCD"] > 0.95).any(axis=1),
    "2 boosted jets (>250, >200), XbbvsQCDTop > 0.95": (df["ak8FatJetPt"][0] > 250)
    & (df["ak8FatJetPt"][1] > 200)
    & (tvars["XbbvsQCDTop"] > 0.95).any(axis=1),
}

for cat in cats:
    print("\n", cat, "\n")
    trig_table = pd.DataFrame(index=list(boostedsels.keys()))
    mask = triggers_dict[cat]["mask"]

    for tkey, tsel in triggers_dict[f"{cat}_minus"]["triggers"].items():
        effs = []
        for bkey, sel in boostedsels.items():
            eff = np.sum(mask & sel & tsel) / np.sum(mask & sel)
            effs.append(f"{eff * 100:.1f}")

        ttkey = tkey.replace("- ", "-") if tkey.startswith("-") else "All"
        trig_table[ttkey] = effs

    sel_effs = []
    for bkey, sel in boostedsels.items():
        eff = np.sum(mask & sel) / np.sum(mask)
        sel_effs.append(f"{eff * 100:.1f}")
    trig_table["Preselection"] = sel_effs

    trig_table.to_csv(plot_dir / f"trig_effs_{cat}.csv")
    print(trig_table.to_markdown(index=True))

2022



 tauhh 

|                                                 |   All |   -PNetBB |   -PNetTauTau |   -PFJet425 |   -Quad-jet |   -SingleTau |   -Di-tau |   Preselection |
|:------------------------------------------------|------:|----------:|--------------:|------------:|------------:|-------------:|----------:|---------------:|
| 1 boosted jet (> 250)                           |  80.9 |      74.3 |          79.8 |        80.7 |        74.3 |         73.8 |      72.4 |           77.2 |
| 2 boosted jets (> 250)                          |  94.4 |      90.9 |          93   |        94.2 |        92.3 |         87.6 |      92.8 |           23   |
| 2 boosted jets (>250, >230)                     |  93.1 |      89.4 |          91.5 |        92.9 |        90.4 |         84.7 |      90.8 |           29.4 |
| 2 boosted jets (>250, >200)                     |  90   |      85.6 |          88.4 |        89.8 |        85.8 |         81   |      86.2 |           39.8 |
| 2 boosted jets (>250, >200), XbbvsQCD > 0.95    |  99.1 |      94.4 |          99   |        99.1 |        97.4 |         97.9 |      98.5 |            6.1 |
| 2 boosted jets (>250, >200), XbbvsQCDTop > 0.95 |  99.3 |      95.2 |          99.2 |        99.3 |        97.6 |         98.8 |      98.9 |            4.6 |

 tauhmu 

|                                                 |   All |   -PNetBB |   -PNetTauTau |   -Muon |   -Mu-tau |   -SingleTau |   -Di-tau |   -PFJet425 |   Preselection |
|:------------------------------------------------|------:|----------:|--------------:|--------:|----------:|-------------:|----------:|------------:|---------------:|
| 1 boosted jet (> 250)                           |  80.7 |      71.2 |          80.1 |    70.7 |      79.7 |         75.5 |      79.8 |        80.6 |           77.1 |
| 2 boosted jets (> 250)                          |  94   |      88   |          92.9 |    90.3 |      93.5 |         87.3 |      93.7 |        93.7 |           17.5 |
| 2 boosted jets (>250, >230)                     |  92   |      86.1 |          90.9 |    87.5 |      91.5 |         84.1 |      91.6 |        91.8 |           22.9 |
| 2 boosted jets (>250, >200)                     |  89.3 |      82.5 |          88.2 |    83.5 |      88.6 |         81.2 |      88.6 |        89.1 |           31.8 |
| 2 boosted jets (>250, >200), XbbvsQCD > 0.95    |  98.2 |      88   |          97.9 |    96.9 |      98.2 |         96.7 |      98.2 |        98.1 |            5.1 |
| 2 boosted jets (>250, >200), XbbvsQCDTop > 0.95 |  98.4 |      88.7 |          98.3 |    97.5 |      98.4 |         97.3 |      98.4 |        98.4 |            3.9 |

 tauhe 

|                                                 |   All |   -PNetBB |   -PNetTauTau |   -EGamma |   -e-tau |   -SingleTau |   -Di-tau |   -PFJet425 |   Preselection |
|:------------------------------------------------|------:|----------:|--------------:|----------:|---------:|-------------:|----------:|------------:|---------------:|
| 1 boosted jet (> 250)                           |  79.3 |      67.9 |          78.8 |      71.1 |     79.2 |         74.8 |      76.6 |        79.2 |           77.2 |
| 2 boosted jets (> 250)                          |  93.1 |      85   |          92.4 |      90.9 |     93.1 |         87.7 |      92.1 |        92.7 |           17.2 |
| 2 boosted jets (>250, >230)                     |  91.4 |      83.5 |          90.6 |      88.4 |     91.4 |         85.1 |      90.4 |        91.2 |           22.4 |
| 2 boosted jets (>250, >200)                     |  88.7 |      80.1 |          87.9 |      85.2 |     88.6 |         81.7 |      87.3 |        88.5 |           31.7 |
| 2 boosted jets (>250, >200), XbbvsQCD > 0.95    |  97.7 |      84.9 |          97.5 |      96.9 |     97.7 |         96.8 |      97.6 |        97.6 |            5.2 |
| 2 boosted jets (>250, >200), XbbvsQCDTop > 0.95 |  97.8 |      85.1 |          97.6 |      97.3 |     97.8 |         97   |      97.8 |        97.7 |            3.9 |


2022EE

tauhh 

|                                                 |   All |   -PNetBB |   -PNetTauTau |   -PFJet425 |   -Quad-jet |   -SingleTau |   -Di-tau |   Preselection |
|:------------------------------------------------|------:|----------:|--------------:|------------:|------------:|-------------:|----------:|---------------:|
| 1 boosted jet (> 250)                           |  80.6 |      73.9 |          79.5 |        80.5 |        74   |         73.4 |      71.9 |           77   |
| 2 boosted jets (> 250)                          |  94.5 |      90.9 |          92.9 |        94.3 |        92.5 |         87.3 |      92.7 |           23   |
| 2 boosted jets (>250, >230)                     |  92.7 |      88.9 |          91   |        92.5 |        89.9 |         84.1 |      90.3 |           29.4 |
| 2 boosted jets (>250, >200)                     |  89.6 |      85.1 |          88   |        89.4 |        85.6 |         80.3 |      85.7 |           39.7 |
| 2 boosted jets (>250, >200), XbbvsQCD > 0.95    |  98.8 |      94.3 |          98.7 |        98.8 |        97.1 |         97.8 |      98.5 |            6.2 |
| 2 boosted jets (>250, >200), XbbvsQCDTop > 0.95 |  99.2 |      95.2 |          99   |        99.2 |        97.6 |         98.3 |      98.9 |            4.7 |

 tauhmu 

|                                                 |   All |   -PNetBB |   -PNetTauTau |   -Muon |   -Mu-tau |   -SingleTau |   -Di-tau |   -PFJet425 |   Preselection |
|:------------------------------------------------|------:|----------:|--------------:|--------:|----------:|-------------:|----------:|------------:|---------------:|
| 1 boosted jet (> 250)                           |  81.1 |      70.9 |          80.5 |    71.3 |      80   |         76   |      80.2 |        81   |           76.9 |
| 2 boosted jets (> 250)                          |  94   |      87.4 |          93   |    90.6 |      93.9 |         87.5 |      93.7 |        93.7 |           17   |
| 2 boosted jets (>250, >230)                     |  92   |      85.4 |          90.8 |    87.6 |      91.7 |         84.4 |      91.6 |        91.8 |           22.6 |
| 2 boosted jets (>250, >200)                     |  89   |      81.6 |          87.9 |    83.2 |      88.4 |         81   |      88.5 |        88.8 |           31.8 |
| 2 boosted jets (>250, >200), XbbvsQCD > 0.95    |  97.8 |      87.2 |          97.6 |    96.5 |      97.7 |         96.8 |      97.5 |        97.7 |            5.1 |
| 2 boosted jets (>250, >200), XbbvsQCDTop > 0.95 |  98.2 |      87.9 |          98   |    97.4 |      98.2 |         97.3 |      98.1 |        98.2 |            3.8 |

 tauhe 

|                                                 |   All |   -PNetBB |   -PNetTauTau |   -EGamma |   -e-tau |   -SingleTau |   -Di-tau |   -PFJet425 |   Preselection |
|:------------------------------------------------|------:|----------:|--------------:|----------:|---------:|-------------:|----------:|------------:|---------------:|
| 1 boosted jet (> 250)                           |  78.8 |      66.9 |          78.2 |      70.7 |     78.6 |         74.3 |      76   |        78.7 |           77   |
| 2 boosted jets (> 250)                          |  93.3 |      85.4 |          92.5 |      91   |     93.2 |         87.4 |      92.3 |        92.9 |           17   |
| 2 boosted jets (>250, >230)                     |  91.3 |      83.5 |          90.3 |      88.5 |     91.2 |         84.4 |      90.2 |        91   |           22.3 |
| 2 boosted jets (>250, >200)                     |  88.1 |      79.7 |          87.1 |      84.5 |     88   |         81   |      86.5 |        87.9 |           31.5 |
| 2 boosted jets (>250, >200), XbbvsQCD > 0.95    |  97.7 |      85.4 |          97.6 |      97.2 |     97.7 |         96.6 |      97.3 |        97.7 |            5   |
| 2 boosted jets (>250, >200), XbbvsQCDTop > 0.95 |  98   |      86   |          97.9 |      97.5 |     98   |         97   |      97.7 |        98   |            3.9 |


In [ ]:
mask = triggers_dict["tauhh"]["mask"]
print(triggers_dict["tauhh"]["triggers"].keys())
print(np.sum(triggers_dict["tauhh"]["triggers"]["PNetBB | PFJet425 | Di-tau"][mask]) / np.sum(mask))
print(
    np.sum(((triggers_dict["tauhh"]["triggers"]["PNetBB | PFJet425 | Di-tau"]) * twoboosted)[mask])
    / np.sum((mask * twoboosted))
)

In [ ]:
print(len(twoboosted))

print(len(triggers_dict["tauhh"]["triggers"]["PNetBB | PFJet425 | Di-tau"]))

In [ ]:
mask = triggers_dict["tauhmu"]["mask"]
print(
    np.sum(triggers_dict["tauhmu"]["triggers"]["ParticleNetBB | IsoMu24 | Mu50 | Mu-tau+Jet"][mask])
    / np.sum(mask)
)
print(
    np.sum(
        (
            (triggers_dict["tauhmu"]["triggers"]["ParticleNetBB | IsoMu24 | Mu50 | Mu-tau+Jet"])
            * twoboosted
        )[mask]
    )
    / np.sum((mask * twoboosted))
)

In [ ]:
mask = triggers_dict["tauhe"]["mask"]
print(
    np.sum(triggers_dict["tauhe"]["triggers"]["ParticleNetBB | Electron | E-tau"][mask])
    / np.sum(mask)
)
print(
    np.sum(
        ((triggers_dict["tauhe"]["triggers"]["ParticleNetBB | Electron | E-tau"]) * twoboosted)[
            mask
        ]
    )
    / np.sum((mask * twoboosted))
)

4b triggers

In [ ]:
plt.rcParams.update({"font.size": 14})

plot_vars = [
    (mhh, "mhh", r"$m_{HH}$ [GeV]", np.linspace(250, 1500, 30)),
    (higgs[:, 0].pt, "hbbpt1", r"H$_1$ $p_{T}$ [GeV]", np.linspace(230, 500, 20)),
    (higgs[:, 1].pt, "hbbpt2", r"H$_2$ $p_{T}$ [GeV]", np.linspace(230, 500, 20)),
]

cat = "bbbb"
(mask, label, triggers) = (
    np.ones(len(df4b)).astype(bool),
    "HH4b",
    {"ParticleNetBB | PFJet425 ": bbbb_triggered},
)


for kinvar, kinname, kinlabel, bins in plot_vars:
    fig, (ax, rax) = plt.subplots(
        2, 1, figsize=(12, 14), gridspec_kw=dict(height_ratios=[4, 1], hspace=0.07), sharex=True
    )

    hists = {"Preselection": np.histogram(kinvar[mask], bins=bins, weights=weights[mask])}
    ratios = {}

    hep.histplot(
        hists["Preselection"],
        yerr=False,
        label="Preselection",
        ax=ax,
    )

    colours = plt.cm.tab10.colors[1:]

    for key, c in zip(triggers.keys(), colours):
        # print(key)
        hists[key] = np.histogram(
            kinvar[mask & triggers[key]], bins=bins, weights=weights[mask & triggers[key]]
        )
        ratios[key] = hists[key][0] / hists["Preselection"][0]

        hep.histplot(
            hists[key],
            yerr=False,
            label=key,
            ax=ax,
            color=c,
        )

        hep.histplot(
            (ratios[key], bins),
            yerr=False,
            label=key,
            ax=rax,
            histtype="errorbar",
            color=c,
            # markersize=20,
            linestyle="--",
            # markeredgewidth=3,
            # fillstyle='none',
        )

    ax.set_ylabel("Events [A.U.]")
    ax.legend()
    ax.set_title(label)
    ax.set_xlim(bins[0], bins[-1])
    ax.set_ylim(0)

    # rax.legend()
    rax.grid(axis="y")
    rax.set_xlabel(kinlabel)
    rax.set_ylabel("Triggered / Preselection")

    ylims = [0.5, 1] if (cat.endswith("minus") and kinname != "mhh") else [0, 1]
    rax.set_ylim(ylims)

    hep.cms.label(ax=ax, data=False, year=2022, com="13.6")

    plt.savefig(plot_dir / f"{kinname}_{cat}.pdf", bbox_inches="tight")
    plt.show()

In [ ]:
plt.rcParams.update({"font.size": 14})


fig, (ax, rax) = plt.subplots(
    2, 1, figsize=(12, 14), gridspec_kw=dict(height_ratios=[4, 1], hspace=0.07), sharex=True
)

bins = np.linspace(250, 1500, 30)
hists = {"Preselection": np.histogram(mhh[mask], bins=bins, weights=weights[mask])}
ratios = {}

hep.histplot(
    hists["Preselection"],
    yerr=False,
    label="Preselection",
    ax=ax,
)

colours = plt.cm.tab10.colors[1:]

for key, c in zip(triggers.keys(), colours):
    print(key)
    hists[key] = np.histogram(
        mhh[mask & triggers[key]], bins=bins, weights=weights[mask & triggers[key]]
    )
    ratios[key] = hists[key][0] / hists["Preselection"][0]

    hep.histplot(
        hists[key],
        yerr=False,
        label=key,
        ax=ax,
        color=c,
    )

    hep.histplot(
        (ratios[key], bins),
        yerr=False,
        label=key,
        ax=rax,
        histtype="errorbar",
        color=c,
        capsize=4,
    )

ax.set_ylabel("Events [A.U.]")
ax.legend()
ax.set_title(label)

rax.legend()
rax.grid(axis="y")
rax.set_xlabel(r"$m_{HH}$ [GeV]")
rax.set_ylabel("Triggered / Preselection")
rax.set_ylim(0, 1)

hep.cms.label(ax=ax, data=False, year=2022, com="13.6")

plt.savefig(plot_dir / f"mhh_{cat}.pdf", bbox_inches="tight")
plt.show()

In [ ]:
ptaus = df["ak8FatJetParTXtauhtauh"] + df["ak8FatJetParTXtauhtaue"] + df["ak8FatJetParTXtauhtaum"]
plt.hist(ptaus[1], np.linspace(0, 1, 100))
# df["ak8FatJetParTXtauhtauh"]